In [37]:
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [38]:
features = pd.read_csv("../Concrete.csv")[:1000]
features.head()

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [39]:
labels = features.pop("csMPa")
labels.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: csMPa, dtype: float64

In [40]:
features = np.array(features)
features = StandardScaler().fit_transform(features)

features = torch.tensor(features, dtype=torch.float32)
labels = torch.tensor(np.array(labels).reshape(-1,1), dtype=torch.float32)

print(features.shape, labels.shape)

torch.Size([1000, 8]) torch.Size([1000, 1])


In [41]:
model = torch.nn.Sequential(
    torch.nn.Linear(8,16),
    torch.nn.LeakyReLU(), # activation functions
    torch.nn.Linear(16,8),
    torch.nn.ReLU(),
    torch.nn.Linear(8,8),
    torch.nn.ReLU(),
    torch.nn.Linear(8,1)
)

preds = model(features)
print(preds.shape)

print(preds[:5])

torch.Size([1000, 1])
tensor([[-0.2387],
        [-0.2322],
        [-0.2194],
        [-0.1630],
        [-0.2117]], grad_fn=<SliceBackward0>)


In [42]:
from torch.nn.functional import mse_loss

loss = mse_loss(preds, labels)
loss

tensor(1577.6868, grad_fn=<MseLossBackward0>)

In [43]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [46]:
epochs = 40000
for epoch in range(epochs):
    preds = model(features)
    loss = mse_loss(preds, labels)
    
    if (epoch+1)%(epochs//10) == 0:
        print(loss)

    loss.backward()
    
    optimizer.step()
    optimizer.zero_grad()

tensor(10.6641, grad_fn=<MseLossBackward0>)
tensor(9.8848, grad_fn=<MseLossBackward0>)
tensor(10.2634, grad_fn=<MseLossBackward0>)
tensor(10.4634, grad_fn=<MseLossBackward0>)
tensor(10.2718, grad_fn=<MseLossBackward0>)
tensor(9.6778, grad_fn=<MseLossBackward0>)
tensor(8.9588, grad_fn=<MseLossBackward0>)
tensor(9.8797, grad_fn=<MseLossBackward0>)
tensor(10.2676, grad_fn=<MseLossBackward0>)
tensor(10.3946, grad_fn=<MseLossBackward0>)


In [50]:
i = 7
preds = model(features[i])
print(preds, labels[i])

tensor([34.1590], grad_fn=<AddBackward0>) tensor([36.4500])
